In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'd95c2a35c31e490ea230fa0f173209aa'

CLIENT_SECRET = '07f336c3653f4ec5825844f9b7259f18'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
import pandas as pd

In [3]:
import spotipy

In [4]:
from spotipy.oauth2 import SpotifyOAuth

In [5]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [8]:
playlist_id = '3wVf2PduEomefWHxyzAQy9'

music_df = get_trending_playlist_data(playlist_id, access_token)
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Ennodu Nee Irundhaal,"A.R. Rahman, Sid Sriram, Sunitha Sarathy",I (Original Motion Picture Soundtrack),1fYpftlxL5gVVKIz8kZ8lr,7mLEUzAulFygMchoGMrP8E,71,2014-09-18,352711,False,https://open.spotify.com/track/7mLEUzAulFygMch...,...,0.603,0,-6.733,1,0.0367,0.5580,0.000000,0.1840,0.207,158.125
1,Innum Konjam Naeram,"A.R. Rahman, Vijay Prakash, Shweta Mohan",Maryan (Original Motion Picture Soundtrack),4NHRsm0K6U2PhxFgNhEBby,5KJSI1MjQeXEBHBYMjgP41,69,2013-05-13,313533,False,https://open.spotify.com/track/5KJSI1MjQeXEBHB...,...,0.389,5,-10.350,1,0.0336,0.5090,0.000008,0.1030,0.436,134.976
2,Hosanna,"A.R. Rahman, Vijay Prakash, Suzanne D'Mello, B...",Vinnathaandi Varuvaayaa (Original Motion Pictu...,1VAR4nGvZh6KW3PLioooYM,3l7JaDIJmWMpvcJqhTJjrk,66,2010-01-08,330547,False,https://open.spotify.com/track/3l7JaDIJmWMpvcJ...,...,0.812,7,-6.147,1,0.0418,0.1660,0.001620,0.0407,0.700,86.051
3,Pudhu Vellai Mazhai,"Unni Menon, Sujatha",Roja,1yhVLCFqvjIAARRbzbhY30,32j96Gl0BjQHAVdMPxin43,4,2014-08-01,313028,False,https://open.spotify.com/track/32j96Gl0BjQHAVd...,...,0.414,0,-9.245,1,0.0426,0.4160,0.000021,0.1170,0.150,61.016
4,Thalli Pogathey,"A.R. Rahman, Sid Sriram, Aaryan Dinesh Kanagar...",Achcham Yenbadhu Madamaiyada (Original Motion ...,3aqLIJUbfDODyEmvpi5Fni,6wBXSf1chpnyXGOSvANs0a,8,2016-06-20,268056,False,https://open.spotify.com/track/6wBXSf1chpnyXGO...,...,0.748,9,-6.029,0,0.0349,0.1080,0.000000,0.0909,0.148,98.228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vennilave Vennilave Vinnai,"Hariharan, Sadhana Sargam",Minsara Kanavu (Original Motion Picture Soundt...,0lu9EJwZFYWUogxPtptP3N,1Q3oZnMXBy55s8uFJmR1bE,0,1997-01-01,351720,False,https://open.spotify.com/track/1Q3oZnMXBy55s8u...,...,0.319,1,-11.631,1,0.0320,0.4310,0.000002,0.1540,0.610,91.024
96,Poo Pookkum Osai Adhai,"M. Vasudevan, Sujatha",Minsara Kanavu (Original Motion Picture Soundt...,0lu9EJwZFYWUogxPtptP3N,32A7ZQqmiIuZDE2y0hFQS3,0,1997-01-01,404307,False,https://open.spotify.com/track/32A7ZQqmiIuZDE2...,...,0.585,2,-12.624,1,0.0754,0.0322,0.000013,0.0940,0.707,100.000
97,"Thanga Thamari Magale (From ""Minsara Kanavu"")",S. P. Balasubrahmanyam,SPB - The Voice For A Millennium,4DAg6C54e0PNteCP5EEqq9,67WU7tXaqhaka2Q2MyTXzb,0,2020-06-02,302708,False,https://open.spotify.com/track/67WU7tXaqhaka2Q...,...,0.489,11,-11.116,0,0.1710,0.0597,0.000053,0.0301,0.867,172.113
98,Roja Roja,"A.R. Rahman, Hariharan",Dil Hi Dil Mein (Original Motion Picture Sound...,15wpQtYDmZO2sFuKG5RUDM,2TIBW1u8JcBXvn4Th2S9MZ,57,1999-10-19,346667,False,https://open.spotify.com/track/2TIBW1u8JcBXvn4...,...,0.417,2,-18.771,1,0.0507,0.0643,0.000001,0.2680,0.418,89.993


In [9]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

# Building Model

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity


data = music_df

In [11]:
# function to calculate weighted popularity based on release date
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    time_span = datetime.now() - release_date

    weight = 1/(time_span.days + 1)
    return weight

In [12]:
# Normalize music features using MinMaxScaler
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
music_features_scaled

array([[0.        , 0.65024903, 0.        , ..., 0.21957483, 0.18519318,
        0.85257279],
       [0.54674221, 0.41339236, 0.45454545, ..., 0.10400913, 0.43233326,
        0.69643809],
       [0.45467422, 0.88157167, 0.63636364, ..., 0.01512341, 0.71724585,
        0.36645016],
       ...,
       [0.66288952, 0.52407305, 1.        , ..., 0.        , 0.89747464,
        0.94691865],
       [0.59206799, 0.44438296, 0.18181818, ..., 0.33942074, 0.4129074 ,
        0.39303805],
       [0.38810198, 0.9346984 , 1.        , ..., 0.17819946, 0.64817613,
        0.94632511]])

# Music Recommendations based on audio features

In [14]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

# Music Recommendations based on weighted Popularity and combine it with content based recommendations

In [20]:
# Assuming content_based_recommendations and calculate_weighted_popularity are defined elsewhere

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Create a DataFrame for the input song
    input_song_data = {
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    }
    input_song_df = pd.DataFrame(input_song_data)

    # Concatenate content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = pd.concat([content_based_rec, input_song_df], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations



In [22]:
input_song_name = 'Ennodu Nee Irundhaal'
recommendations = hybrid_recommendations(input_song_name)
print(f"Hybrid recommendation songs for '{input_song_name}':")
recommendations

Hybrid recommendation songs for 'Ennodu Nee Irundhaal':


,Track Name,Artists,Album Name,Release Date,Popularity
0,Mannipaaya,"A.R. Rahman, Shreya Ghoshal",Vinnathaandi Varuvaayaa (Original Motion Pictu...,2010-01-08,63.0
3,En Kadhale,"S. P. Balasubrahmanyam, Noel James",Duet,1994-05-20,57.0
4,Anjali Anjali,"S. P. Balasubrahmanyam, K. S. Chithra",Duet,1994-05-20,57.0
1,En Meal Vizhundha Mazhai,"P. Jayachandran, K. S. Chithra",May Madham,2021-07-25,52.0
2,Thamizha Thamizha,Hariharan,Roja,2014-08-01,0.0


In [23]:
hybrid_recommendations('Pudhu Vellai Mazhai')

,Track Name,Artists,Album Name,Release Date,Popularity
4,Pookkalae Sattru Oyivedungal,"A.R. Rahman, Haricharan, Shreya Ghoshal",I (Original Motion Picture Soundtrack),2014-09-18,62.0
1,Mazhai Kuruvi,A.R. Rahman,Chekka Chivantha Vaanam (Original Motion Pictu...,2018-09-28,60.0
0,Adiye,"A.R. Rahman, Sid Sriram",Kadal (Original Motion Picture Soundtrack),2012-12-14,51.0
2,Ennavale Adi Ennavale,Unnikrishnan,Kaadhalan,2014-08-01,36.0
3,Vennilave Vennilave Vinnai,"Hariharan, Sadhana Sargam",Minsara Kanavu (Original Motion Picture Soundt...,1997-01-01,0.0


In [25]:
content_based_recommendations('Pudhu Vellai Mazhai')

,Track Name,Artists,Album Name,Release Date,Popularity
71,Adiye,"A.R. Rahman, Sid Sriram",Kadal (Original Motion Picture Soundtrack),2012-12-14,51
28,Mazhai Kuruvi,A.R. Rahman,Chekka Chivantha Vaanam (Original Motion Pictu...,2018-09-28,60
16,Ennavale Adi Ennavale,Unnikrishnan,Kaadhalan,2014-08-01,36
95,Vennilave Vennilave Vinnai,"Hariharan, Sadhana Sargam",Minsara Kanavu (Original Motion Picture Soundt...,1997-01-01,0
20,Pookkalae Sattru Oyivedungal,"A.R. Rahman, Haricharan, Shreya Ghoshal",I (Original Motion Picture Soundtrack),2014-09-18,62


#  Cosine similarity is a metric used to measure the similarity between two vectors of real numbers. In the context of machine learning, it is often employed to determine the similarity between documents or feature vectors. In scikit-learn, the cosine_similarity function is available to compute cosine similarity between samples in a given dataset.
# Formula for cosine Similarity
# A.B / ||A||.||B||
## where A and B are vectors and ||A|| and ||B|| are the euclidean's norm of the vectoes A and B